# SCE-TTS: HiFi-GAN 학습

## 1. 구글 드라이브 마운트

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

## 2. 필수 라이브러리 및 함수 불러오기

In [ ]:
import sys
from pathlib import Path

In [ ]:
%cd /content
!git clone --depth 1 https://github.com/sce-tts/TTS.git -b sce-tts
%cd /content/TTS
!python setup.py develop

## 3. 학습할 데이터셋 불러오기

In [ ]:
%cd /content/TTS
!cp "/content/drive/My Drive/Colab Notebooks/data/filelists.zip" ./filelists.zip
!rm -rf ./filelists
!unzip -q filelists.zip -d ./filelists

## 4. 사전 학습 데이터 불러오기

In [ ]:
%cd /content/TTS
!mkdir -p "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2"
if not Path("/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/config.json").exists():
    !gdown --id 1vRxp1RH-U7gSzWgyxnKY4h_7pB3tjPmU -O hifigan-v2.zip
    !unzip -q hifigan-v2.zip -d ./
    !cp -R ./hifigan-v2/* "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/"

In [ ]:
%cd /content/TTS
if not Path("/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/scale_stats_new.npy").exists():
    !python TTS/bin/compute_statistics.py "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/config.json" "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/scale_stats_new.npy" --data_path "/content/TTS/filelists/wavs/"

## 5. TensorBoard 실행

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="/content/drive/My Drive/Colab Notebooks/data/hifigan-v2"

## 6. HiFi-GAN 학습 진행

In [ ]:
# 처음학습을 진행할 때는 코드를 그대로 실행하면 된다.
# 이어서 진행을 하고 싶으면 2~3번째 줄의 코드 주석을 해제하고 3번째 줄에 이어서 학습진행할 모델의 경로를 입력해준다.
# 그리고 4번째 줄 밑의 코드를 모두 지워줍니다.

%cd /content/TTS
# !(python TTS/bin/train_vocoder_gan.py \
#     --continue_path "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/CONTINUE_DIRECTORY")
!(python TTS/bin/train_vocoder_gan.py \
    --config_path "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/config.json" \
    --coqpit.data_path "/content/TTS/filelists/wavs"  \
    --coqpit.audio.stats_path "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/scale_stats_new.npy"  \
    --coqpit.output_path "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/"  \
    --coqpit.num_loader_workers 2  \
    --coqpit.num_val_loader_workers 2  \
    --restore_path "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/model_file.pth.tar")